In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

import fastai
from fastai import *
from fastai.vision import *
from fastai.callbacks.tensorboard import *
from fastai.vision.gan import *
from superRes.generators import *
from superRes.critics import *
from superRes.dataset import *
from superRes.loss import *
from superRes.save import *
from PIL import Image, ImageDraw, ImageFont
from PIL import ImageFile

import torchvision

import geffnet # efficient/ mobile net

In [2]:
def get_data(bs:int, sz:int, keep_pct:float):
    return get_databunch(sz=sz, bs=bs, crappy_path=path_lowRes, 
                         good_path=path_fullRes, 
                         random_seed=None, keep_pct=keep_pct)

def create_training_images(fn, i, p_hr, p_lr, size):
    """Create low quality images from folder p_hr in p_lr"""
    dest = p_lr/fn.relative_to(p_hr)
    dest.parent.mkdir(parents=True, exist_ok=True)
    img = PIL.Image.open(fn)
    targ_sz = resize_to(img, size, use_min=True)
    img = img.resize(targ_sz, resample=PIL.Image.BILINEAR).convert('RGB')
    img.save(dest, quality=60) 

In [5]:
path = untar_data(URLs.PETS)

path_fullRes = path/'images'
path_lowRes = path/'lowRes-96'
path_medRes = path/'lowRes-256'

proj_id = 'unet_superRes'

gen_name = proj_id + '_gen'
crit_name = proj_id + '_crit'

TENSORBOARD_PATH = Path('data/tensorboard/' + proj_id)

nf_factor = 2
pct_start = 1e-8

In [13]:
bs=25
sz=128
lr = 1e-3
wd = 1e-3
keep_pct=1.0
epochs = 10

In [14]:
data_gen = get_data(bs=bs, sz=sz, keep_pct=keep_pct)

In [28]:
res = models.resnet34

print(res.__module__)
help(res)

torchvision.models.resnet
Help on function resnet34 in module torchvision.models.resnet:

resnet34(pretrained=False, progress=True, **kwargs)
    ResNet-34 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr



In [7]:
shuffle = models.shufflenet_v2_x2_0 #non è supportato da unet_learner
print(type(shuffle))
# dir(shuffle)

<class 'function'>


In [21]:
efficient = geffnet.efficientnet_b5
print(type(efficient))
help(efficient)

<class 'function'>
Help on function efficientnet_b5 in module geffnet.gen_efficientnet:

efficientnet_b5(pretrained=False, **kwargs)
    EfficientNet-B5



In [9]:
mobile = geffnet.mobilenetv3_100
print(type(mobile))
help(mobile)

<class 'function'>
Help on function mobilenetv3_100 in module geffnet.mobilenetv3:

mobilenetv3_100(pretrained=False, **kwargs)
    MobileNet V3



In [10]:
help(cnn_learner)

Help on function cnn_learner in module fastai.vision.learner:

cnn_learner(data:fastai.basic_data.DataBunch, base_arch:Callable, cut:Union[int, Callable]=None, pretrained:bool=True, lin_ftrs:Union[Collection[int], NoneType]=None, ps:Union[float, Collection[float]]=0.5, custom_head:Union[torch.nn.modules.module.Module, NoneType]=None, split_on:Union[Callable, Collection[Collection[torch.nn.modules.module.Module]], NoneType]=None, bn_final:bool=False, init=<function kaiming_normal_ at 0x7f079bed5e18>, concat_pool:bool=True, **kwargs:Any) -> fastai.basic_train.Learner
    Build convnet style learner.



In [29]:
cnn = cnn_learner(data=data_gen, base_arch=res)

In [30]:
cnn.summary()

Sequential
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               [64, 64, 64]         9,408      False     
______________________________________________________________________
BatchNorm2d          [64, 64, 64]         128        True      
______________________________________________________________________
ReLU                 [64, 64, 64]         0          False     
______________________________________________________________________
MaxPool2d            [64, 32, 32]         0          False     
______________________________________________________________________
Conv2d               [64, 32, 32]         36,864     False     
______________________________________________________________________
BatchNorm2d          [64, 32, 32]         128        True      
______________________________________________________________________
ReLU                 [64, 32, 32]         0          False     
___________________________________________________

In [31]:
learn_gen = gen_learner_deep(data=data_gen, arch=res, gen_loss=FeatureLoss(), nf_factor=nf_factor)

learn_gen.summary()

DynamicUnetDeep
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               [64, 64, 64]         9,408      False     
______________________________________________________________________
BatchNorm2d          [64, 64, 64]         128        True      
______________________________________________________________________
ReLU                 [64, 64, 64]         0          False     
______________________________________________________________________
MaxPool2d            [64, 32, 32]         0          False     
______________________________________________________________________
Conv2d               [64, 32, 32]         36,864     False     
______________________________________________________________________
BatchNorm2d          [64, 32, 32]         128        True      
______________________________________________________________________
ReLU                 [64, 32, 32]         0          False     
______________________________________________